In [4]:
import numpy as np
from tqdm.notebook import tqdm
import open3d as o3d
import matplotlib.pyplot as plt
import pickle
from cylinder_fitting import fit
from cylinder_fitting import fitting_rmsd
from myShape3d import Plane, Sphere

### Run Test Set

#### Cylinder - Cone

In [ ]:
# def fitting_rmsd(w_fit, C_fit, r_fit, Xs):
#     '''Calculate the RMSD of fitting.'''
#     return np.sqrt(sum((geometry.point_line_distance(p, C_fit, w_fit) - r_fit) ** 2
#                     for p in Xs) / len(Xs)) 

test_label = pickle.load(open('../pkl/GroundTruth.pkl', 'rb'))

N = 915
START = 1

indexCylinder = []
indexCone = []
errCone = []
errCylinder = []
rmsdNormCone = []
rmsdNormCylinder = []
stdCone = []
stdCylinder = []

for i in tqdm(range(START, START + N)):
    label = test_label[i]
    pcd_load = o3d.io.read_point_cloud(f'../ply/test/pointCloud/pointCloud{i}.ply')
    pts = np.asarray(pcd_load.points)

    try:
        if label == 'cylinder': # cylinder
            rf = open(f'../ply/test/GTpointCloud/{i}_2.txt', "w")

            axis, center, radius, fit_err = fit(pts)

            vecC_stakado = np.stack([axis] * len(pts), 0)
            dist_pt = np.cross(vecC_stakado, (center - pts))
            dist_pt = np.linalg.norm(dist_pt, axis=1)
            l = np.abs(dist_pt - radius) / radius
            
            indexCylinder.append(i)
            errCylinder.append(fit_err)
            rmsdNormCylinder.append(fitting_rmsd(axis, center, radius, pts))
            stdCylinder.append(np.std(l))
            

            rf.write('2\n')

            rf.write(str(np.format_float_positional(radius, precision=9, trim='-')) + '\n')

            for i in axis:
                rf.write(str(np.format_float_positional(i, precision=9, trim='-')))
                rf.write('\n')
            
            
            for i in center:
                rf.write(str(np.format_float_positional(i, precision=9, trim='-')))
                rf.write('\n')

            rf.close()

        if label == 'cone':
            rf = open(f'../ply/test/GTpointCloud/{i}_4.txt', "w")

            axis, center, radius, fit_err = fit(pts)
            vecC_stakado = np.stack([axis] * len(pts), 0)
            dist_pt = np.cross(vecC_stakado, (center - pts))
            dist_pt = np.linalg.norm(dist_pt, axis=1)
            l = np.abs(dist_pt - radius) / radius

            indexCone.append(i)
            errCone.append(fit_err)
            rmsdNormCone.append(fitting_rmsd(axis, center, radius, pts))
            stdCone.append(np.std(l))

            rf.write('4\n')

            rf.write(str(np.format_float_positional(radius, precision=9, trim='-')) + '\n')

            for i in axis:
                rf.write(str(np.format_float_positional(i, precision=9, trim='-')))
                rf.write('\n')
            
            
            for i in center:
                rf.write(str(np.format_float_positional(i, precision=9, trim='-')))
                rf.write('\n')

            rf.close()
    except:
        print('FAIL:', i)
        break
        
    

# print('index cylinder:', indexCylinder)
# print('index cone:', indexCone)

# print('err cylinder:', errCylinder)
# print('err cone:', errCone)

# print('rmsd cylinder:', rmsdNormCylinder)
# print('rmsd cone:', rmsdNormCone)

# print('std cylinder:', stdCylinder)
# print('std cone:', stdCone)

#### Plane, Sphere, Torus

In [8]:
import pickle
from tqdm.notebook import tqdm
import numpy as np
import open3d as o3d
import geom3d, fit3d


test_label = pickle.load(open('../pkl/GroundTruth.pkl', 'rb'))

N = 925
START = 1

indexPlane = []
inliersPlane = []
indexSphere = []
inliersSphere = []
indexTorus = []


for i in tqdm(range(START, START + N)):
    label = test_label[i]
    pcd_load = o3d.io.read_point_cloud(f'../ply/test/pointCloud/pointCloud{i}.ply')
    pts = np.asarray(pcd_load.points)

    try:
        if label == 'plane':
            rf = open(f'../ply/test/GTpointCloud/{i}_1.txt', "w")

            plane = Plane()
            normal, center, loss, inliers = plane.fit(pts)

            indexPlane.append(i)
            inliersPlane.append(len(inliers) / len(pts))

            rf.write('1\n')

            for i in normal:
                rf.write(str(np.format_float_positional(i, precision=9, trim='-')))
                rf.write('\n')

            for i in center:
                rf.write(str(np.format_float_positional(i, precision=9, trim='-')))
                rf.write('\n')

            rf.close()

        if label == 'sphere':
            rf = open(f'../ply/test/GTpointCloud/{i}_3.txt', "w")

            sphere = Sphere()
            center, radius, inliers = sphere.fit(pts, k=0.3, maxIteration=100)

            indexSphere.append(i)
            inliersSphere.append(len(inliers) / len(pts))

            rf.write('3\n')

            rf.write(str(np.format_float_positional(radius, precision=9, trim='-')) + '\n')

            for i in center:
                rf.write(str(np.format_float_positional(i, precision=9, trim='-')))
                rf.write('\n')

            rf.close()

        if label == 'torus':
            rf = open(f'../ply/test/GTpointCloud/{i}_5.txt', "w")

            # c_init = np.mean(pts, axis=0)
            # initial_guess = geom3d.Torus([c_init[0], c_init[1], c_init[2]], [0, 0, 1], 1, 0.1)
            initial_guess = geom3d.Torus([0, 0, 0], [0, 0, 1], 1, 0.1)
            torus = fit3d.torus_fit(pts, initial_guess=initial_guess)
            axis = np.array(torus.direction)
            axis = axis / np.linalg.norm(axis)
            center = np.array(torus.center)
            majorRadius = torus.major_radius
            minorRadius = torus.minor_radius

            indexTorus.append(i)

            rf.write('5\n')

            rf.write(str(np.format_float_positional(majorRadius, precision=9, trim='-')) + '\n')
            rf.write(str(np.format_float_positional(minorRadius, precision=9, trim='-')) + '\n')

            for i in axis:
                rf.write(str(np.format_float_positional(i, precision=9, trim='-')))
                rf.write('\n')

            for i in center:
                rf.write(str(np.format_float_positional(i, precision=9, trim='-')))
                rf.write('\n')

            rf.close()

    except:
        print('FAIL:', i)
        break

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 